In [206]:
#imports
import pulp as p
from pulp import value
import numpy as np
import matplotlib as plt

In [283]:
#fixed values
#HIER muss noch alles auf englisch beschrieben werden, variablen auch elleganter umgenannt werden
#ERINNERUNG, wenn Werte BSPW. im 15min Takt vorliegen müssen auch die Leistungswerte der Wärmepumpe und Pufferspeicher durch 4 geteilt werden (soll alles automatisiert werden)


#Strompreise müssten hier eigentlich auch durch 4 geteilt werden
strompreise = [
    0.2909, 0.2981, 0.3053, 0.3124, 0.3194, 0.3263, 0.3331, 0.3396, 0.3460, 0.3521,
    0.3579, 0.3635, 0.3687, 0.3736, 0.3781, 0.3822, 0.3860, 0.3893, 0.3922, 0.3947,
    0.3967, 0.3982, 0.3993, 0.3999, 0.4000, 0.3996, 0.3988, 0.3975, 0.3957, 0.3935,
    0.3908, 0.3877, 0.3842, 0.3802, 0.3759, 0.3712, 0.3661, 0.3607, 0.3550, 0.3491,
    0.3428, 0.3364, 0.3297, 0.3229, 0.3159, 0.3089, 0.3017, 0.2945, 0.2873, 0.2801,
    0.2730, 0.2659, 0.2589, 0.2521, 0.2454, 0.2390, 0.2328, 0.2268, 0.2211, 0.2157,
    0.2106, 0.2059, 0.2016, 0.1977, 0.1941, 0.1910, 0.1883, 0.1861, 0.1843, 0.1830,
    0.1822, 0.1818, 0.1820, 0.1825, 0.1836, 0.1852, 0.1872, 0.1896, 0.1925, 0.1958,
    0.1996, 0.2037, 0.2082, 0.2131, 0.2184, 0.2239, 0.2297, 0.2358, 0.2422, 0.2487,
    0.2555, 0.2624, 0.2694, 0.2765, 0.2837, 0.2909
]

temperatur = [
    7.4, 7.4, 7.3, 7.3, 7.2, 7.2, 7.1, 7.1, 7.0, 6.9, 6.9, 6.8, 6.8, 6.7, 6.6, 6.6,
    6.5, 6.5, 6.4, 6.3, 6.3, 6.2, 6.1, 6.1, 11.5, 11.7, 12.0, 12.2, 12.4, 12.6, 12.8,
    13.0, 13.3, 13.4, 13.6, 13.8, 14.0, 14.1, 14.3, 14.4, 14.5, 14.6, 14.7, 14.8, 14.9,
    14.9, 15.0, 14.9, 14.9, 14.8, 14.7, 14.6, 14.5, 14.4, 14.3, 14.1, 14.0, 13.8, 13.6,
    13.4, 13.3, 13.0, 12.8, 12.6, 12.4, 12.2, 12.0, 11.7, 11.5, 6.0, 6.1, 6.1, 6.2,
    6.3, 6.3, 6.4, 6.5, 6.5, 6.6, 6.6, 6.7, 6.8, 6.8, 6.9, 6.9, 7.0, 7.1, 7.1, 7.2,
    7.2, 7.3, 7.3, 7.4, 7.2, 7.0, 6.8
]

anschaffungskosten_speicher = 6000
anschaffunskosten_pumpe = 7800
jahresrechnung_heizkosten_vor_pumpe = 2370

T_innen = 20  # Ziel-Innentemperatur in °C
Wohnfläche = 150  # Wohnfläche in m²
Wärmebedarf_pro_m2 = 60  # Wärmebedarf in W/m² bei normaler Dämmung
T_norm_aussen = -12  # Normaußentemperatur (zur Normierung des Temperaturunterschieds) in °C
heizungstemperatur=35 #Fußbodenheizung optimalerweise
wirkungsgrad=0.8
cop_vorfaktor=0.2
Delta_T_max = T_innen - T_norm_aussen

# Maximaler Temperaturunterschied
Delta_T_max = T_innen - T_norm_aussen

# Heizlastberechnung für jede Stunde
heizlast = [(Wärmebedarf_pro_m2 * Wohnfläche * (T_innen - T_außen) / Delta_T_max) for T_außen in temperatur]

# Stundenweise Heizlast in kW umrechnen (von W zu kW)
heizlast_kW = [round(last / 1000, 2) for last in heizlast]

# Berechnung der COP-Werte
t_warm = 273.15 + heizungstemperatur
t_kalt = [x + 273.15 for x in temperatur]
cop_werte = [cop_vorfaktor * (t_warm / (t_warm - kalt)) for kalt in t_kalt]

# Kosten pro kW(h) Wärme berechnen, in diesem Fall kWh, da sich alle Werte auf Stunden beziehen
kosten_pro_kW = [strompreis / cop for cop, strompreis in zip(cop_werte, strompreise)]

print(kosten_pro_kW)



# Muss aus COP/Strompeis gibt mir wieviel Euro es zu Zeit t kostet um einen kW Wärme zu erzeugen
c = kosten_pro_kW

T = len(c)
p_max = 5
s_max = 22
s_laden = 4
s_entladen = 2
d = 15/24 #Heizbedarf in kW pro stunde
b_start = 0

[0.13027486613662187, 0.13349926983611887, 0.13721904916436795, 0.14041018984260906, 0.14407463897452547, 0.14718708421223434, 0.15079490507869533, 0.15373746552003884, 0.15719617069608957, 0.16053886094434544, 0.16318335226350816, 0.1663264643842284, 0.16870582508518575, 0.17155411325653097, 0.17423397695927295, 0.176123316566607, 0.1785007301638812, 0.18002677267564496, 0.18200421872464723, 0.18380480285575201, 0.1847361674509167, 0.18608080480285583, 0.18724273892584764, 0.18752409540808032, 0.15252312185623887, 0.1510738276813241, 0.1488301152036346, 0.1470550056790525, 0.14510498134025654, 0.1430212558818756, 0.14077170209313636, 0.13839688463410676, 0.1352773000162258, 0.13325198766834348, 0.13052506895992197, 0.12768846341067655, 0.1247460652279734, 0.1223207853318188, 0.11923576180431601, 0.1166876521174754, 0.11402563686516307, 0.11135096543890949, 0.10859824760668511, 0.10583449618692191, 0.10302758396884647, 0.10074460490021105, 0.09790686354048353, 0.09604819081616106, 0.09

In [284]:
#Hier Grafik und Herleitung für: Von Wetter und Wärmebedarf zu COP, von COP und Strompreis zu C  

In [285]:
#Hier grafische Erklärung für die Nebenbedingungen

DAS HIER FUNKTIONIERT NOCH NICHT und text muss noch angepasst werden und besser gelayoutet werden
# Zielfunktion und Nebenbedingungen für das Optimierungsproblem

## Zielfunktion
Minimierung der Gesamtkosten:
- \( P_t \): Wärmeerzeugung der Wärmepumpe in Stunde \( t \)
- \( c_t \): Kosteneffizienz (EUR pro kW Wärme) in Stunde \( t \)
- Ziel: Minimierung der Gesamtkosten über alle Stunden

\[
\min \sum_{t=0}^{23} c_t \cdot P_t
\]

---

## Nebenbedingungen

1. **Grenzen der Wärmeerzeugung**
   - Die Wärmeerzeugung \( P_t \) ist durch die maximale Leistung der Wärmepumpe beschränkt:
   \[
   0 \leq P_t \leq P_{\text{max}}
   \]

2. **Grenzen für das Laden und Entladen des Speichers**
   - Einspeicherung (\( x^{\text{in}}_t \)) und Ausspeicherung (\( x^{\text{out}}_t \)) des Speichers sind beschränkt:
   \[
   0 \leq x^{\text{in}}_t \leq X_{\text{in,max}}, \quad 0 \leq x^{\text{out}}_t \leq X_{\text{out,max}}
   \]

3. **Grenzen für den Speicherfüllstand**
   - Der Speicherfüllstand (\( B_t \)) ist durch die Speicherkapazität begrenzt:
   \[
   0 \leq B_t \leq B_{\text{max}}
   \]

4. **Wärmebilanz**
   - Die Wärmenachfrage \( d \) muss durch die Wärmeerzeugung \( P_t \), Einspeicherung (\( x^{\text{in}}_t \)), und Ausspeicherung (\( x^{\text{out}}_t \)) gedeckt werden:
   \[
   P_t + x^{\text{out}}_t = d + x^{\text{in}}_t, \quad \forall t \in \{0, \dots, 23\}
   \]

5. **Dynamik des Speicherfüllstands**
   - Der Speicherfüllstand \( B_t \) entwickelt sich über die Zeit durch Einspeicherung (\( x^{\text{in}}_t \)) und Ausspeicherung (\( x^{\text{out}}_t \)):
   \[
   B_0 = B_{\text{start}} + x^{\text{in}}_0 - x^{\text{out}}_0
   \]
   \[
   B_t = B_{t-1} + x^{\text{in}}_t - x^{\text{out}}_t, \quad \forall t \in \{1, \dots, 23\}
   \]

---

## Notation
- \( P_t \): Wärmeerzeugung (kW)
- \( x^{\text{in}}_t \): Einspeicherung (kW)
- \( x^{\text{out}}_t \): Ausspeicherung (kW)
- \( B_t \): Speicherfüllstand (kWh)
- \( c_t \): Kosteneffizienz (EUR/kW)
- \( d \): Wärmenachfrage (kW)
- \( P_{\text{max}} \): Maximale Wärmeerzeugungskapazität (kW)
- \( X_{\text{in,max}}, X_{\text{out,max}} \): Maximale Lade- und Entladeleistung (kW)
- \( B_{\text{max}} \): Maximale Speicherkapazität (kWh)
- \( B_{\text{start}} \): Anfangsfüllstand des Speichers (kWh)


In [286]:
#initialize model and its direction (memory space to assign objective function and constraints)
model = p.LpProblem("problem", p.LpMinimize)

In [287]:
#decision variables & their bounds / constraints

#creates heat pump variable & adds constraints to it
P = [p.LpVariable(f"p_{t}", lowBound=0, upBound=p_max) for t in range(T)]
#creates thermal storage variable & adds constraints to it
B = [p.LpVariable(f"B_{t}", lowBound=0, upBound=s_max) for t in range(T)]
#creates charge variable & adds constraints to it
x_in = [p.LpVariable(f"x_in_{t}", lowBound=0, upBound=s_laden) for t in range(T)]
#creates discharge variable & adds constraints to it
x_out = [p.LpVariable(f"x_out_{t}", lowBound=0, upBound=s_entladen) for t in range(T)]

In [288]:
#objective function
model += p.lpSum(c[t] * P[t] for t in range(T))

In [289]:
#constraints

#cover heat demand
for t in range(T):
    model += d == P[t] + x_out[t] - x_in[t]
#start level of thermal storage
model += B[0] == b_start + x_in[0] - x_out[0] 
#dynamics of thermal storage
for t in range(1, T):
    model += B[t] == B[t-1] + x_in[t] - x_out[t] 
#noch n ergänzen, also Wärmeverlustquote (nicht nur hier, natürlich auch bei start level)

In [290]:
status = model.solve() 
print(p.LpStatus[status])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/juliandomnik/PycharmProjects/heatpump-operation-optimization/.venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/m4/qcxwdmxd5b94sx7ctbs447dc0000gn/T/e36a6c1e47e34b17ad2e4df91acdd771-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/m4/qcxwdmxd5b94sx7ctbs447dc0000gn/T/e36a6c1e47e34b17ad2e4df91acdd771-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 197 COLUMNS
At line 965 RHS
At line 1158 BOUNDS
At line 1543 ENDATA
Problem MODEL has 192 rows, 384 columns and 671 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 192 (0) rows, 383 (-1) columns and 670 (-1) elements
0  Obj 0 Primal inf 59.999904 (96)
78  Obj 0 Primal inf 59.999904 (96)
156  Obj 3.6743787 Primal inf 666.5249 (96)
234  Obj 6.0430514 Primal inf 1.4749908 (9)
244  Obj 6.0897391
Optimal - object

In [291]:
#optimal values

print("Optimale Ergebnisse:")
for t in range(T):
    print(
        f"Stunde {t}: "
        f"P[t] = {P[t].varValue:.3f}, "
        f"x_in[t] = {x_in[t].varValue:.3f}, "
        f"x_out[t] = {x_out[t].varValue:.3f}, "
        f"B[t] = {B[t].varValue:.3f}"
    )


Optimale Ergebnisse:
Stunde 0: P[t] = 4.625, x_in[t] = 4.000, x_out[t] = 0.000, B[t] = 4.000
Stunde 1: P[t] = 4.625, x_in[t] = 4.000, x_out[t] = 0.000, B[t] = 8.000
Stunde 2: P[t] = 4.625, x_in[t] = 4.000, x_out[t] = 0.000, B[t] = 12.000
Stunde 3: P[t] = 4.625, x_in[t] = 4.000, x_out[t] = 0.000, B[t] = 16.000
Stunde 4: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 0.625, B[t] = 15.375
Stunde 5: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 0.625, B[t] = 14.750
Stunde 6: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 0.625, B[t] = 14.125
Stunde 7: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 0.625, B[t] = 13.500
Stunde 8: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 0.625, B[t] = 12.875
Stunde 9: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 0.625, B[t] = 12.250
Stunde 10: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 0.625, B[t] = 11.625
Stunde 11: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 0.625, B[t] = 11.000
Stunde 12: P[t] = 0.000, x_in[t] = 0.000, x_out[t] = 0.625, B[t] = 10.375
Stunde 13: P[t] = 0.000, x_in

In [292]:
#Hier Grafik für optimalen Betrieb (plt)

In [293]:
#Mini-Exkurs Lineare Algebra / Veranschaulichung des Problems 
print("Modelldimensionen: " + str(len(model.constraints)) + " x "+ str(len(model.variables())))
print("------------------")
print("Anzahl der Constraints (m):", len(model.constraints))
print("Anzahl der Variablen (n):", len(model.variables()))


Modelldimensionen: 192 x 384
------------------
Anzahl der Constraints (m): 192
Anzahl der Variablen (n): 384


# Matrixdarstellung

In der linearen Programmierung gilt folgendes: für jeden Constraint (Gleichung) gibt es eine Zeile in der Matrix \( A \), und für jede Variable gibt es eine Spalte. Die Werte in der Matrix \( A \) geben die Koeffizienten der Variablen in den Constraints an.

Die Matrix \( A \) mit Dimension \( 96 \times 96 \):

\[
A =
\begin{bmatrix}
a_{1,1} & a_{1,2} & a_{1,3} & \cdots & a_{1,M} \\
a_{2,1} & a_{2,2} & a_{2,3} & \cdots & a_{2,M} \\
a_{3,1} & a_{3,2} & a_{3,3} & \cdots & a_{3,M} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
a_{N,1} & a_{N,2} & a_{N,3} & \cdots & a_{N,M} \\
\end{bmatrix}
\]



In [297]:
#Amortisationsdauer der Wärmepumpe alleine
tag_kosten_pumpe_only = sum(d * preis for preis in strompreise)/4

tägliche_ersparnisse = jahresrechnung_heizkosten_vor_pumpe/365 - tag_kosten_pumpe_only

print("Average cost for heating before: ", jahresrechnung_heizkosten_vor_pumpe/365)
print("Cost with WP: ",tag_kosten_pumpe_only)
print("Ersparnisse pro Tag: ", tägliche_ersparnisse)
print("Average savings in %: ", tägliche_ersparnisse/(jahresrechnung_heizkosten_vor_pumpe/365)*100)
print("Amortisationsdauer in Jahren: ", anschaffunskosten_pumpe/tägliche_ersparnisse/365)

Average cost for heating before:  6.493150684931507
Cost with WP:  4.363609375
Ersparnisse pro Tag:  2.129541309931507
Average savings in %:  32.796733254219404
Amortisationsdauer in Jahren:  10.034960540110847


In [295]:
#Amortisationsdauer des Wärmespeichers

# Optimalen Wert der Zielfunktion in einer Variablen speichern
optimal_value = value(model.objective)/4


print("Daily cost with pufferspeicher", optimal_value)
print("Daily cost without pufferspeicher", tag_kosten_pumpe_only)

ersparnisse = tag_kosten_pumpe_only - optimal_value
tage_bis_gelohnt = anschaffungskosten_speicher/ersparnisse

print("Gespart pro Tag in Euro: ", ersparnisse)
print("Amortisationsdauer basierend auf den letzten 24h in Jahren: ", tage_bis_gelohnt/365)

Daily cost with pufferspeicher 1.5224347720266103
Daily cost without pufferspeicher 4.363609375
Gespart pro Tag in Euro:  2.84117460297339
Amortisationsdauert basierend auf den letzten 24h in Tagen:  2111.8026304053215
Amortisationsdauer basierend auf den letzten 24h in Jahren:  5.785760631247456


In [ ]:
#reale Schätzungen für Amortisationsdauer